# Reading OME-NGFF

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ome/EMBL-EBI-imaging-course-05-2023/blob/main/Day_5/Reading_images.ipynb)


## Learning Objectives

* Learn how to access local OME-NGFF file in Python
* Learn how to access remote OME-NGFF file in Python

There are several ways to access data. For the purpose of the topics covered in this workshop, we will access files over ``https`` and use [dask](https://dask.org/)

Some sofware packages required to have all the 2D planes in memory in order to work other can work on planar data. We will now show two mechanisms to access the data depending on the needs using ``dask.array.from_zarr``.

Main point to keep in mind is that
binary data are not loaded until it is used, i.e. it is **lazily loaded**.


## Launch

This notebook uses the ``environment.yml`` file.

See [Setup](./workshop.ipynb).

## How to access local Zarr file

We first look at an existing ``ome.zarr`` file i.e. ``mri.ome.zarr``

In [6]:
import dask
import dask.array as da
from dask.diagnostics import ProgressBar
import numpy
def load_binary_from_local_with_data(path):
    with ProgressBar():
        return numpy.asarray(da.from_zarr(path))

In [7]:
%%time
image_location = 'images/mri.ome.zarr/s0'
data = load_binary_from_local_with_data(image_location)
print(data.shape)

[########################################] | 100% Completed | 105.85 ms
(27, 226, 186)
CPU times: user 29.9 ms, sys: 15.8 ms, total: 45.7 ms
Wall time: 118 ms


**Exercise**: if you have generated a file locally as part of the [conversion workflow](Conversion.ipynb), set the ``image_location`` parameter to, for example, ``/tmp/conversion_out/B4_C3.zarr/0/0``.

In [23]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *

n = 2
if len(data.shape) == 3:
    n = 0
    
def update(z=0):
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    c = 1
    t = 0
    if len(data.shape) == 3: 
        plt.imshow(data[z, :, :])
    else:
        plt.imshow(data[t, c, z, :, :])
    fig.canvas.flush_events()


interact(update, z= widgets.IntSlider(value=0, min=0, max=data.shape[n]-1, step=1, description="Select slice", continuous_update=False))

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select slice', max=256), Output…

<function __main__.update(z=0)>

## How to access Zarr file on S3

We need to specify the ``endpoint`` and the path to the image.

### Install dependencies if required

The cell below will install dependencies if you choose to run the notebook in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true). **Do not run the cell if you are not running the notebook on Google Colab**.


If using Google Colab, **do not** use the ``Runtime>Run all`` entry.

In [ ]:
%pip install aiohttp==3.8.4 zarr==2.14.2

## Read the data
We use the same image as in Day 4. The Tiff image has been converted into OME-Zarr and is available on S3.

In [ ]:
image_id = 6001247

### Option 1
Load the binary. In that case, we load the 5D-image. This might be required depending on the software used to analyse the data. This approach should only be used if the 5D-image is required.

In [16]:
import dask
import dask.array as da
from dask.diagnostics import ProgressBar
import numpy

def load_binary_from_s3_with_data(id, resolution='0'):
    endpoint_url = 'https://uk1s3.embassy.ebi.ac.uk/'
    root = 'idr/zarr/v0.1/%s.zarr/%s/' % (id, resolution)
    with ProgressBar():
        return numpy.asarray(da.from_zarr(endpoint_url + root))

In [17]:
%%time
data = load_binary_from_s3_with_data(image_id)
print(data.shape)

[########################################] | 100% Completed | 10.79 s
(1, 2, 257, 210, 253)
CPU times: user 2.89 s, sys: 669 ms, total: 3.56 s
Wall time: 11.2 s


In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *

def update(z=0):
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    c = 1
    t = 0
    plt.imshow(data[t, c, z, :, :])
    fig.canvas.flush_events()


interact(update, z= widgets.IntSlider(value=0, min=0, max=data.shape[2]-1, step=1, description="Select Z", continuous_update=False))

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Z', max=256), Output()),…

<function __main__.update(z=0)>

### Option 2

The method below will return a dask array **without** any binary data i.e. **lazy loading**. The dimension order of the array returned is ``(TCZYX)``. **Data will be loaded when requested later.**

In [13]:
def load_binary_from_s3(id, resolution='0'):
    endpoint_url = 'https://uk1s3.embassy.ebi.ac.uk/'
    root = 'idr/zarr/v0.1/%s.zarr/%s/' % (id, resolution)
    return da.from_zarr(endpoint_url + root)

In [14]:
%time data = load_binary_from_s3(image_id)

CPU times: user 23.8 ms, sys: 13.7 ms, total: 37.5 ms
Wall time: 280 ms


In [15]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *

def update(z=0):
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    c = 1
    t = 0
    plt.imshow(data[t, c, z, :, :])
    fig.canvas.flush_events()


interact(update, z= widgets.IntSlider(value=0, min=0, max=data.shape[2]-1, step=1, description="Select Z", continuous_update=False))

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Z', max=256), Output()),…

<function __main__.update(z=0)>

### License (BSD 2-Clause)
Copyright (C) 2023 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.